<a href="https://colab.research.google.com/github/seyf1elislam/LocalLLM_OneClick_Colab/blob/main/Run_any_gguf_model_in_TextGen_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $\color{#19ABEA}{\text{Run any gguf quantized models in Latest Version TextGen-webui :}}$
### `version 05/2024`
> changes : uses hf client  (5/2024)

this [Notebook](https://github.com/seyf1elislam/LocalLLM_OneClick_Colab) allow u run  any quantized gguf file from any Hugging face repository as long as it fits the colab Vram and ram.

>follow this [githubRepository](https://github.com/seyf1elislam/LocalLLM_OneClick_Colab) to get the updates of this noteboook.

> Created by :
> 😇 github :   [@seyf1elislam](https://github.com/seyf1elislam)
> 🤗 Huggingface : [@seyf1elislam](https://huggingface.co/seyf1elislam)

## where can i find gguf quantized models ?
> you can check those 🤗 huggingface repos :
- [Thebloke](https://huggingface.co/TheBloke) #1 best gguf source ,contains hundreds of quantized models(gptq ,gguf ,awQ).
- [LongStriker](https://huggingface.co/LoneStriker) #2 best gguf source ,contains hundreds of quantized models.(exl2,gguf)
- [seyf1elislam](https://huggingface.co/seyf1elislam) contains some gguf models.  
- [using search](https://huggingface.co/models?sort=trending&search=gguf) here u can find all gguf files on  huggingface.

## need good models to try ?
> you can try those :  
- [Meta-Llama-3-8B-Instruct-GGUF](https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF)   (recommended  🔥) .
- [seyf1elislam/WestKunai-Hermes-7b-GGUF](https://huggingface.co/seyf1elislam/WestKunai-Hermes-7b-GGUF)
- [TheBloke/SOLAR-10.7B-v1.0-GGUF](https://huggingface.co/TheBloke/SOLAR-10.7B-v1.0-GGUF)
- [TheBloke/Kunoichi-7B-GGUF](https://huggingface.co/TheBloke/Kunoichi-7B-GGUF)
- [TheBloke/Starling-LM-7B-alpha-GGUF](https://huggingface.co/TheBloke/Starling-LM-7B-alpha-GGUF)
-[seyf1elislam/KuTrix-7b-GGUF](https://huggingface.co/seyf1elislam/KuTrix-7b-GGUF)
-[seyf1elislam/KunaiBeagle-Hermes-7b](https://huggingface.co/seyf1elislam/KunaiBeagle-Hermes-7b)

## some Tips ?
> most mistral models goes with 8k context length if u want to use longer context u need to make sure the models support longer context.

> if u want to run model higher then 13B (such as 20B,4x7b..) on colab u may need to reduce the offloaded gpu models to split the ram usage between gpu vram and system ram. (slower but it works 😉)

In [ ]:
#@title # $\color{#19ABEA}{\text{Download and install requirements}}$ {display-mode: "form"}
#@markdown $\color{#19ABEA}{\text{run this only once at the start}}$

## @markdown `Note :no need to check these checkboxes unless u facing a problem`
#@markdown ---
#@markdown `check this if u need to change the version only`
delete_existing_texgen_webui_folder =False #@param {type:"boolean"}
if delete_existing_texgen_webui_folder :
  !rm -rf /content/text-generation-webui
#@markdown ---
#@markdown ### TextGen webui version :
#@markdown `note : gemma models are supported in snapshot-2024-02-25 or newer`
%cd /content
branch = "dev" # @param ["dev", "snapshot-2024-01-07", "snapshot-2024-01-14", "snapshot-2024-01-28", "snapshot-2024-02-11", "snapshot-2024-02-25", "snapshot-2024-03-03", "snapshot-2024-03-10", "snapshot-2024-03-17", "snapshot-2024-03-24", "snapshot-2024-03-31", "snapshot-2024-04-07", "snapshot-2024-04-14", "snapshot-2024-04-21", "snapshot-2024-04-28"]
branch = f"-b {branch}"
#@markdown `to download the latest version of TextGen Webui use use_latest ,this will ignore the branch name.`
use_latest =False #@param {type:"boolean"}

if use_latest :
  branch = ""
!git clone {branch} https://github.com/oobabooga/text-generation-webui

%cd /content/text-generation-webui
!pip install -r requirements.txt
!pip install git+https://github.com/UWUplus/flask-cloudflared
!pip install -r extensions/openai/requirements.txt --upgrade
!apt-get -y install -qq aria2
#@markdown ----
reinstall_llama_cpp_python = False # @param {type:"boolean"}
if reinstall_llama_cpp_python :
  !pip uninstall -y llama-cpp-python
  !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir


reinstall_flash_attention = False # @param {type:"boolean"}
if reinstall_flash_attention:
  !pip uninstall -y flash-attn
  !pip install --no-build-isolation flash-attn==2.3.0
#@markdown ----

from IPython.display import clear_output
clear_output()
print("Done")

In [ ]:
#@title #  $\color{#19ABEA}{Download Model}$ {display-mode: "form"}
#@markdown ## $\color{#19ABEA}{\text{Copy the RepoName here :}}$

#----------
from huggingface_hub import HfFileSystem
fs = HfFileSystem()

def get_donwloadlink_and_filename(repo_name,quant='Q5_K_M'):
  gguf_files = fs.glob(f"{repo_name}/**.gguf")
  filtered_files = [file for file in gguf_files if (quant in file or (quant.lower()) or  (quant.upper()) in file)]

  if len(filtered_files) ==0 : # quant not existed
    print( "=" * 10 +"\navailable quants:\n",gguf_files)
    return None,None
  file_path= filtered_files[0]
  *repo_name , file_name = file_path.split("/");
  repo_name='/'.join(repo_name)
  return f"https://huggingface.co/{repo_name}/resolve/main/{file_name}?download=true" ,file_name

#----------

#@markdown  Example :`seyf1elislam/WestKunai-Hermes-7b-GGUF`
repo_name = "QuantFactory/Meta-Llama-3-8B-Instruct-GGUF" # @param ["seyf1elislam/WestKunai-Hermes-7b-GGUF", "seyf1elislam/KuTrix-7b-GGUF", "TheBloke/Mistral-7B-OpenOrca-GGUF", "TheBloke/zephyr-7B-alpha-GGUF", "TheBloke/zephyr-7B-beta-GGUF", "TheBloke/neural-chat-7B-v3-1-GGUF", "TheBloke/mistral-ft-optimized-1218-GGUF", "TheBloke/xDAN-L1-Chat-RL-v1-GGUF", "TheBloke/openchat-3.5-1210-starling-slerp-GGUF", "TheBloke/Loyal-Macaroni-Maid-7B-GGUF", "TheBloke/Valkyrie-V1-GGUF", "TheBloke/Sonya-7B-GGUF", "TheBloke/Starling-LM-7B-alpha-GGUF", "TheBloke/SOLAR-10.7B-v1.0-GGUF", "QuantFactory/Meta-Llama-3-8B-Instruct-GGUF"] {allow-input: true}

#@markdown  in colab gpu (15Gvram) you can use :
#@markdown  - 13b model quantized upto Q5_K_M(`context up to 4K`)
#@markdown  - 7b model quantized upto Q8_0(`context up to 16K or even more if ur model stable with long context`)
quant ="Q5_K_M" #@param ['Q2_K', 'Q3_K_L', 'Q3_K_M', 'Q3_K_S', 'Q4_0', 'Q4_1', 'Q4_K_M', 'Q4_K_S', 'Q5_0', 'Q5_1', 'Q5_K_M', 'Q5_K_S', 'Q6_K', 'Q8_0']
model_download_url,model_file_name =get_donwloadlink_and_filename(repo_name,quant=quant)

if model_download_url is not None and model_file_name is not None :
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_download_url} -d /content/text-generation-webui/models/ -o {model_file_name}
else :
  print("======================")
  print("please check the repo name or check availability of the quantized file inside the repo")

In [ ]:
#@title ##  $\color{#19ABEA}{RunOnly}$ {display-mode: "form"}
# %cd /content/text-generation-webui
#@markdown  $\color{#19ABEA}{\text{Enter ur Context size or let it unchecked to use the default size}}$
params =" "
#@markdown _____
api =True # @param {type:"boolean"}
if api :params +=" --api --public-api --listen "
api_only_mode_nowebui =False # @param {type:"boolean"}
if api_only_mode_nowebui : params+=" --nowebui "
#@markdown _____
use_custom_context_len = True # @param {type:"boolean"}
context_len=8192 #@param
if use_custom_context_len : params += f" --n_ctx  {context_len} "
#@markdown _____
use_custom_params = True # @param {type:"boolean"}
custom_params = "" # @param {type:"string"}
if use_custom_params : params +=custom_params

#@markdown _____
# params
!python server.py   --share  --n-gpu-layers 100000  	--model $model_file_name   $params